In [14]:
import pandas as pd
import yfinance as yf
import re
import numpy as np
from termcolor import colored
import sys


def get_currency(x):
    if pd.notna(x) and x != "":
        return yf.Ticker(x).basic_info["currency"]
    else:
        return ""


class Portfolio:
    def __init__(self, path=sys.path[0] + "/portfolio.csv",):
        self.path = path
        self.df_raw = pd.read_csv(path, usecols=["Investment","Quantity"], index_col=False)
        self.usd_gbp = yf.Ticker("USDGBP=X")
        self.cash = {"GBP": self.df_raw.iloc[-1]["Quantity"],
                     "USD": round(self.df_raw.iloc[-1]["Quantity"] * (1 / self.usd_gbp.basic_info["previousClose"]), 2)}


    def __repr__(self):
        return self.df_stocks.to_string()
    

    def _init_stocks_df(self):
        self.df_stocks = self.df_raw.copy().iloc[0:-1]

        self.df_stocks["Exchange"] = self.df_stocks["Investment"].apply(lambda x: re.search(r"\((.*?)\:", x).group()[1:-1] if re.search(r"\((.*?)\:", x) else "")
        self.df_stocks["Ticker"] = self.df_stocks["Investment"].apply(lambda x: re.search(r"\:(.*?)\)", x).group()[1:-1] if re.search(r"\((.*?)\:", x) else "")
        self.df_stocks["Ticker"] = self.df_stocks["Ticker"] + self.df_stocks["Exchange"].apply(lambda x: ".L" if x == "LSE" else "")
        
        self.df_stocks["Currency"] = ""
        self.df_stocks["Currency"] = self.df_stocks["Ticker"].where(self.df_stocks["Ticker"] != "").apply(get_currency)


        self.df_stocks["Price Raw"] = self.df_stocks["Ticker"].apply(lambda x: yf.Ticker(x).basic_info["previousClose"])
        
        self.df_stocks["Price GBP"] = np.nan
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "GBP", self.df_stocks["Price Raw"]).round(2)
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "USD", self.df_stocks["Price Raw"] * self.usd_gbp.basic_info["bid"]).round(2)
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "GBp", self.df_stocks["Price Raw"] / 100).round(2)
    
        self.df_stocks["Price USD"] = (self.df_stocks["Price GBP"] * (1 / self.usd_gbp.basic_info["bid"])).round(2)

        self.df_stocks["Total Value GBP"] = self.df_stocks["Price GBP"] * self.df_stocks["Quantity"]
        self.df_stocks["Total Value USD"] = self.df_stocks["Price USD"] * self.df_stocks["Quantity"]


    def show_portoflio_totals(self):
        self.total_gbp = self.df_stocks['Total Value GBP'].sum() + self.cash['GBP']
        self.total_usd = self.df_stocks['Total Value USD'].sum() + self.cash['USD']
        print("Portfolio totals:")
        print(colored(f"£ {self.total_gbp:,.2f}", "light_green"))
        print(colored(f"$ {self.total_usd:,.2f}", "light_green"))


    def show_forecast(self, years):
        print()
        print(f"{years}Y forecast:")
        print(f"£ {round(self.total_gbp * 1.09**years, 2):,.2f}")
        print(f"$ {round(self.total_usd * 1.09**years, 2):,.2f}")

In [18]:

x = yf.Ticker("AAPL").history()
for i, val in enumerate(x.items()):
    print(i, val)



0 ('Open', Date
2024-03-11 00:00:00-04:00    172.940002
2024-03-12 00:00:00-04:00    173.149994
2024-03-13 00:00:00-04:00    172.770004
2024-03-14 00:00:00-04:00    172.910004
2024-03-15 00:00:00-04:00    171.169998
2024-03-18 00:00:00-04:00    175.570007
2024-03-19 00:00:00-04:00    174.339996
2024-03-20 00:00:00-04:00    175.720001
2024-03-21 00:00:00-04:00    177.050003
2024-03-22 00:00:00-04:00    171.759995
2024-03-25 00:00:00-04:00    170.570007
2024-03-26 00:00:00-04:00    170.000000
2024-03-27 00:00:00-04:00    170.410004
2024-03-28 00:00:00-04:00    171.750000
2024-04-01 00:00:00-04:00    171.190002
2024-04-02 00:00:00-04:00    169.080002
2024-04-03 00:00:00-04:00    168.789993
2024-04-04 00:00:00-04:00    170.289993
2024-04-05 00:00:00-04:00    169.589996
2024-04-08 00:00:00-04:00    169.029999
2024-04-09 00:00:00-04:00    168.692001
Name: Open, dtype: float64)
1 ('High', Date
2024-03-11 00:00:00-04:00    174.380005
2024-03-12 00:00:00-04:00    174.029999
2024-03-13 00:00:00-